In [4]:
#in the dataset i added titles to all the different columns
import pandas as pd 
import re

df = pd.read_csv("training_data.csv", encoding='latin-1')

df2 = df.sample(n = 1500, random_state = 30)
df3 = df2[['text','Target']]

p1 = re.compile(r"@\w+\s")
p2 = re.compile(r"\W+")
tweet = df3['text']
tweet = tweet.replace({p1: ""}, regex=True)
tweet = tweet.replace({p2: " "}, regex=True)
df3['text'] = tweet

print(df3)


                                                      text  Target
218153                     no 3g wireless in farm country        0
865260                    thank you for your warm welcome        4
526545   twitterfox is addictive CASS you did not quote...       0
351397   i hate thunder x 156873582395823678 ugh whyyyy...       0
641337   Actual last day of college Pity it has to fini...       0
327040                               omgg thts so luckyyy        0
180515                          Is not feeling well again        0
675273   going to summer school on cruches and my compu...       0
500353                                           is tired        0
871672   going to my orientation and then baby sitting ...       4
959683   lol that helps too but i have a sexy mcfly mug xx       4
989230   its near twister everone is moving but the ren...       4
1037868                  hello thank you for following me        4
491529   these people are driving me crazy Im a human b...    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
df3.loc[df3['Target'] > 3, 'Target'] = 1
df3.loc[df3['Target'] < 1, 'Target'] = -1
df3.loc[df3['Target'] == 2, 'Target'] = 0

print(df3)

#data has been cleaned and formatted

                                                      text  Target
218153                     no 3g wireless in farm country       -1
865260                    thank you for your warm welcome        1
526545   twitterfox is addictive CASS you did not quote...      -1
351397   i hate thunder x 156873582395823678 ugh whyyyy...      -1
641337   Actual last day of college Pity it has to fini...      -1
327040                               omgg thts so luckyyy       -1
180515                          Is not feeling well again       -1
675273   going to summer school on cruches and my compu...      -1
500353                                           is tired       -1
871672   going to my orientation and then baby sitting ...       1
959683   lol that helps too but i have a sexy mcfly mug xx       1
989230   its near twister everone is moving but the ren...       1
1037868                  hello thank you for following me        1
491529   these people are driving me crazy Im a human b...    

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
#making train and test data
#x = independant which in this case is the column 'text'
#y = depedant which in this case is the column 'Target'
from sklearn.model_selection import train_test_split
X = df3['text']
y = df3['Target']
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size = 0.20, random_state=30)


In [7]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score



In [8]:
def tokenize(text):     
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

en_stopwords = set(line.strip() for line in open("stopwords.txt")) 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1),
    stop_words = en_stopwords)


In [9]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=30)
print(kfolds)

StratifiedKFold(n_splits=5, random_state=30, shuffle=True)


In [10]:
np.random.seed(1)

pipeline_svm = make_pipeline(vectorizer, 
                            SVC(probability=True, kernel="linear", class_weight="balanced"))

grid_svm = GridSearchCV(pipeline_svm,
                    param_grid = {'svc__C': [0.01, 0.1, 1, 10]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm.fit(X_train, y_train)
grid_svm.score(X_test, y_test)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   17.7s finished


0.7350701800847458

In [11]:
grid_svm.best_params_
#best c to use for SVM tuning

{'svc__C': 0.1}

In [16]:
grid_svm.predict(["ew this sucks"])

array([-1])

In [ ]:
## knn